In [1]:
import numpy as np
import pandas as pd
import os.path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import DenseNet121  # Import DenseNet model
from tensorflow.keras.applications import EfficientNetB0  # Import DenseNet model
import random
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Dense, MaxPooling2D, AveragePooling2D, Flatten, Concatenate
from tensorflow.keras.models import Model
from keras.models import load_model
import os
import cv2
from keras import layers, Input, models, optimizers, losses
from tensorflow.keras.models import save_model

os.chdir(r'/kaggle/working')
os.listdir('/kaggle/input')



counter = 0

def roi_clahe_pre_process(path, output_directory):
    global counter  # Access the global counter variable

    img = cv2.imread(path)

    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply thresholding
    thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    # Determine bounding rectangle of the thresholded image
    x, y, w, h = cv2.boundingRect(thresh)
    x, y, w, h = x, y, w + 20, h + 20

    # Crop the original image based on the bounding rectangle
    img = img[y:y+h, x:x+w]

    # Convert the cropped image to grayscale
    gray_crop = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Apply CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    clahe_img = clahe.apply(gray_crop)

    # Construct the output file path
    filename = os.path.basename(path)
    output_filename = filename[:-4] + '__enhanced_' + str(counter) + filename[-4:]
    output_path = os.path.join(output_directory, output_filename)

    # Save the output image
    cv2.imwrite(output_path, clahe_img)
    # Increment the counter
    counter += 1

    return output_path


def create_dataset(path,output_directory, num_fractured=12000, num_normal=12000):
    dataset = []
    fractured_count = 0
    normal_count = 0

    for id_p in os.listdir(path):
        patient_id = id_p
        path_id = path + '/' + str(id_p)
        for lab in os.listdir(path_id):
            if lab.split('_')[-1] == 'positive':
                label = 1
            elif lab.split('_')[-1] == 'negative':
                label = 0
            path_l = path_id + '/' + str(lab)
            for img in os.listdir(path_l):
                img_path = os.path.join(path_l, img)
                img_path = roi_clahe_pre_process(img_path, output_directory)  # Apply CLAHE
                if label == 1 and fractured_count < num_fractured:
                    fractured_count += 1
                    dataset.append({
                        'label': label,
                        'image_path': img_path
                    })
                if label == 0 and normal_count < num_normal:
                    normal_count += 1
                    dataset.append({
                        'label': label,
                        'image_path': img_path
                    })
    
    random.shuffle(dataset)
    return dataset

# Load dataset into train and test df
train_dir = ('/kaggle/input/better-mura/MURA-v1.1/MURA-v1.1/train_augmented/XR_ELBOW')
output_directory = '/kaggle/working'
traindata = create_dataset(train_dir,output_directory)
trainfilepaths = []
trainlabels = []


for row in traindata:
    trainfilepaths.append(row['image_path'])
    trainlabels.append(row['label'])

trainfilepaths = pd.Series(trainfilepaths, name='Filepath').astype(str)
trainlabels = pd.Series(trainlabels, name='Label').astype(str)
train_df = pd.concat([trainfilepaths, trainlabels], axis=1)

test_dir = ('/kaggle/input/better-mura/MURA-v1.1/MURA-v1.1/valid/XR_ELBOW')
testdata = create_dataset(test_dir,output_directory)
testfilepaths = []
testlabels = []

for row in testdata:
    testlabels.append(row['label'])
    testfilepaths.append(row['image_path'])

testfilepaths = pd.Series(testfilepaths, name='Filepath').astype(str)
testlabels = pd.Series(testlabels, name='Label').astype(str)
test_df = pd.concat([testfilepaths, testlabels], axis=1)

#ImageDataGenerator for data augmentation and preprocessing
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    validation_split=0.2)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input)
# Data generators
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=False
)

2024-04-09 13:35:02.416427: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-09 13:35:02.416532: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-09 13:35:02.570295: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 19200 validated image filenames belonging to 2 classes.
Found 4800 validated image filenames belonging to 2 classes.
Found 465 validated image filenames belonging to 2 classes.


In [2]:
pretrained_model = tf.keras.applications.resnet50.ResNet50(input_shape=(224, 224, 3), include_top=False, weights='imagenet', pooling='avg')

inputs = pretrained_model.input
x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
x = tf.keras.layers.Dense(50, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs, outputs)

# Adam optimizer with low learning rate for better accuracy
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
class CustomModelCheckpoint(tf.keras.callbacks.ModelCheckpoint):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        filepath = self.filepath.format(epoch=epoch + 1, **logs)
        if self.save_best_only:
            current = logs.get(self.monitor)
            if current is None:
                warnings.warn(f'Can save best model only with {self.monitor} available, skipping.')
            else:
                if self.monitor_op(current, self.best):
                    if self.verbose > 0:
                        print(f'\nEpoch {epoch + 1}: {self.monitor} improved from {self.best:.5f} to {current:.5f}, saving model to {filepath}')
                    self.best = current
                    if self.save_weights_only:
                        self.model.save(filepath, overwrite=True)
                    else:
                        self.model.save(filepath, overwrite=True)
        else:
            if self.verbose > 0:
                print(f'\nEpoch {epoch + 1}: saving model to {filepath}')
            if self.save_weights_only:
                self.model.save(filepath, overwrite=True)
            else:
                self.model.save(filepath, overwrite=True)

# Define checkpoint path with a placeholder for epoch and metric values
checkpoint_path = "/kaggle/working/Resnet50_{epoch:02d}_{val_loss:.4f}.keras"
# Define early stopping callback
callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

learn_control = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.2, min_lr=1e-7)

# Create a custom model checkpoint callback
model_checkpoint = CustomModelCheckpoint(
                        checkpoint_path,
                        monitor='val_loss',
                        save_best_only=True,
                        save_weights_only=False,  # Save the whole model
                        mode='min',
                        verbose=1
                    )

# Train model
print("-------Training Elbow" + "-------")
history = model.fit(train_images, batch_size=32, validation_data=val_images, epochs=60, callbacks=[callbacks, model_checkpoint, learn_control])

# Evaluate model
results = model.evaluate(test_images, verbose=1)
print("First Evaluation")
print("Elbow Results:")
print(results)
print(f"Test Accuracy: {np.round(results[1] * 100, 2)}%")

-------Training Elbow-------
Epoch 1/60


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1712670225.909161      73 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1712670226.008819      73 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 287ms/step - accuracy: 0.7678 - loss: 0.4508

W0000 00:00:1712670402.962651      73 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update



Epoch 1: val_loss improved from inf to 0.53578, saving model to /kaggle/working/Resnet50_01_0.5358.keras
600/600 ━━━━━━━━━━━━━━━━━━━━ 270s 320ms/step - accuracy: 0.7679 - loss: 0.4507 - val_accuracy: 0.8081 - val_loss: 0.5358 - learning_rate: 1.0000e-04
Epoch 2/60
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step - accuracy: 0.9221 - loss: 0.1886
Epoch 2: val_loss improved from 0.53578 to 0.26979, saving model to /kaggle/working/Resnet50_02_0.2698.keras
600/600 ━━━━━━━━━━━━━━━━━━━━ 189s 313ms/step - accuracy: 0.9221 - loss: 0.1886 - val_accuracy: 0.8875 - val_loss: 0.2698 - learning_rate: 1.0000e-04
Epoch 3/60
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 289ms/step - accuracy: 0.9615 - loss: 0.0968
Epoch 3: val_loss improved from 0.26979 to 0.21927, saving model to /kaggle/working/Resnet50_03_0.2193.keras
600/600 ━━━━━━━━━━━━━━━━━━━━ 188s 313ms/step - accuracy: 0.9615 - loss: 0.0968 - val_accuracy: 0.9250 - val_loss: 0.2193 - learning_rate: 1.0000e-04
Epoch 4/60
600/600 ━━━━━━━━━━━━━━━━━━━━ 187s 311ms/st

W0000 00:00:1712672678.821315      74 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


In [4]:
results = model.evaluate(test_images, verbose=1)
print("Second Evaluation")
print("Elbow Results 2:")
print(results)
print(f"Test Accuracy 2: {np.round(results[1] * 100, 2)}%")

15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 120ms/step - accuracy: 0.8663 - loss: 0.6177
Second Evaluation
Elbow Results 2:
[0.676069438457489, 0.8559139966964722]
Test Accuracy 2: 85.59%


In [5]:
modelpath="/kaggle/working/ResNet50.keras"
model.save(modelpath)

In [6]:
Elbow_Model = load_model(modelpath)
results = model.evaluate(test_images, verbose=1)
print("Loaded Model Results")
print("Elbow Results:")
print(results)
print(f"Test Accuracy: {np.round(results[1] * 100, 2)}%")

15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 118ms/step - accuracy: 0.8663 - loss: 0.6177
Loaded Model Results
Elbow Results:
[0.676069438457489, 0.8559139966964722]
Test Accuracy: 85.59%


In [7]:
import os
from keras.models import load_model

# Directory containing the model files
model_directory = '/kaggle/working'

# Get a list of all .keras files in the model directory
model_files = [f for f in os.listdir(model_directory) if f.endswith('.keras')]

# Iterate over each model file
for model_file in model_files:
    # Load the model
    loaded_model = load_model(os.path.join(model_directory, model_file))
    
    # Evaluate the model on the test dataset
    results = loaded_model.evaluate(test_images, verbose=1)
    
    # Print the results
    print(f"Model: {model_file}")
    print("Test Results:")
    print(f"Loss: {results[0]}")
    print(f"Accuracy: {results[1]}")
    print()

 2/15 ━━━━━━━━━━━━━━━━━━━━ 1s 118ms/step - accuracy: 0.8906 - loss: 0.4575

W0000 00:00:1712672720.647498      76 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


15/15 ━━━━━━━━━━━━━━━━━━━━ 8s 235ms/step - accuracy: 0.8663 - loss: 0.6177


W0000 00:00:1712672724.001993      75 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Model: ResNet50.keras
Test Results:
Loss: 0.676069438457489
Accuracy: 0.8559139966964722

 2/15 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step - accuracy: 0.7969 - loss: 0.7234

...

W0000 00:00:1712672808.753048      75 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Model: Resnet50_05_0.1982.keras
Test Results:
Loss: 0.6153351068496704
Accuracy: 0.8301075100898743

 1/15 ━━━━━━━━━━━━━━━━━━━━ 1:14 5s/step - accuracy: 0.9062 - loss: 0.4779

W0000 00:00:1712672826.448365      75 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


15/15 ━━━━━━━━━━━━━━━━━━━━ 9s 258ms/step - accuracy: 0.8663 - loss: 0.6177
Model: Resnet50_08_0.1705.keras
Test Results:
Loss: 0.676069438457489
Accuracy: 0.8559139966964722



W0000 00:00:1712672830.114584      73 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
